In [ ]:
#Residual: output of previous layer, is added to the output of the current layer.
#the doted lines in the paper image correspond to the section were a spatial adjustment has been made before the add t0
#to match all the outputs.

In [6]:
import torch
import torch.nn as nn

from functools import partial
from dataclasses import dataclass
from collections import OrderedDict

In [ ]:
nn.Conv2d()

In [8]:
class Conv2dAuto(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding =  (self.kernel_size[0] // 2, self.kernel_size[1] // 2) # dynamic add padding based on the kernel_size

        #the partial function sets the parameters before the function has been call with no option to modify it.        
conv3x3 = partial(Conv2dAuto, kernel_size=3, bias=False)   

In [9]:
conv = conv3x3(in_channels=32, out_channels=64)
print(conv)
del conv
#como podemos fer en el output los valores de kernel size es igual al definido en la parte de partial, el pading
#es igual a 1 porque es el default en la funcion Conv2d y el bias es igual a false porque lo hemos definido nostros 
#anteriormente

Conv2dAuto(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


In [10]:
def activation_func(activation):
    return  nn.ModuleDict([
        ['relu', nn.ReLU(inplace=True)],
        ['leaky_relu', nn.LeakyReLU(negative_slope=0.01, inplace=True)],
        ['selu', nn.SELU(inplace=True)],
        ['none', nn.Identity()]
    ])[activation]

In [16]:
activations = activation_func('selu')

In [17]:
#residual block definition:takes as input with in_channels ,applies some block of conv to REDUCE IT to out_channels
#and sum it up to the original output.IF THE SIZE MISSMATCH input goes into an identity.

SELU(inplace=True)

In [18]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, activation='relu'):
        super().__init__()
        self.in_channels, self.out_channels, self.activation = in_channels, out_channels, activation
        self.blocks = nn.Identity()
        self.activate = activation_func(activation)
        self.shortcut = nn.Identity()   
    
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut: residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        x = self.activate(x)
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels